In [1]:
class Circle():
    def __init__(self, radius):
        self.radius = 4
    def area(self, radius = 4):
        return 3.14 * radius**2
    def circumference(self, radius = 4):
        return 2 * radius * 3.14

In [2]:
assert Circle
circ1 = Circle(radius=4)
assert circ1.radius == 4

### BEGIN HIDDEN TESTS
circ2 = Circle(radius=10)
assert circ2.radius == 10

AssertionError: 

In [3]:
circ2.radius

4